In [ ]:
import datetime
import re
import pandas as pd
import numpy as np

import altair as alt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import folium

In [ ]:
def to_epiweek(cal_date: datetime.date):
    delta = cal_date - datetime.date(cal_date.year, 1, 1)
    days = delta.days
    epiweek = (days // 7) + 1
    epiweek = epiweek if epiweek < 53 else 52
    return (cal_date.year, epiweek)

assert to_epiweek(datetime.date(2020, 1, 1)) == (2020, 1)
assert to_epiweek(datetime.date(2020, 12, 31)) == (2020, 52)
assert to_epiweek(datetime.date(2021, 1, 1)) == (2021, 1)

In [ ]:
country_codes = pd.read_csv('data/iso_codes.csv')

In [ ]:
covid19_df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

In [ ]:
covid19_df = pd.merge(covid19_df, country_codes, how='left', left_on='iso_code', right_on='Alpha3')
covid19_df = covid19_df.set_index('iso_code')
covid19_df

In [ ]:
# Collect some filterable values

columns = covid19_df.columns.tolist()
continents = covid19_df.continent.unique()
locations = covid19_df.location.unique()

# Set globals

selected_continent = 'Africa'
selected_country = locations[0]
first_metric = 'new_cases_smoothed'
second_metric = 'new_deaths_smoothed'
time_period = 600

In [ ]:
def filter_countries(continent):
    global selected_continent
    selected_continent = continent
    
    countries = covid19_df.loc[covid19_df.continent == continent]
    countries = countries.location.unique()
    
    global selected_country
    selected_country = countries[0]
    
    return (
        interact(select_country, country = widgets.Dropdown(
            options=countries,
            value=selected_country,
            description='Country',
            disabled=False,
        ))
    )

In [ ]:
def select_country(country):
    global selected_country
    selected_country = country

In [ ]:
def set_first_metric(metric):
    global first_metric
    first_metric = metric

In [ ]:
def set_second_metric(metric):
    global second_metric
    second_metric = metric

In [ ]:
def set_period(period):
    global time_period
    time_period = period

In [ ]:
def draw_chart():
    global selected_country
    global first_metric
    global second_metric
    global time_period
      
    country_df = covid19_df.loc[covid19_df.location == selected_country]
    
    country_df = country_df.tail(time_period)

    base = alt.Chart(country_df, width=1000).encode(
        alt.X('yearmonthdate(date):T', title='epiweek:O', axis=alt.Axis(tickCount=14, labelAngle=90))
    )

    mark_bar = base.mark_bar().encode(
        y = alt.Y(first_metric)
#         color = alt.Color('positive_rate', scale = alt.Scale(range=['blue', 'green', 'yellow', 'red']), title='Case Positivity')
    )

    mark_line = base.mark_line(color='red').encode(
        y = alt.Y(second_metric)
    )

    return alt.layer(mark_bar, mark_line).resolve_scale(y = 'independent')

In [ ]:
# compute when a range is monotonically increasing
def is_increasing(arr):
    for i in range(1, len(arr)):
        if arr[i-1] >= arr[i]:
            return False
    else:
        return True

In [ ]:
def cases_increasing(country_df,increase_window):
    country_df.insert(0, 'increasing', country_df.new_cases_smoothed.rolling(increase_window).apply(is_increasing))
    country_df.insert(0, 'increasing_avg', country_df.increasing.rolling(increase_window).mean())

In [ ]:
def draw_trend_chart():
    global selected_country
    global first_metric
    global second_metric
    global time_period
    
    country_df = covid19_df.loc[covid19_df.location == selected_country]
    
    country_df = country_df.tail(time_period)
    
    increase_window = 7
    
    country_df.insert(0, 'increasing', country_df.new_cases_smoothed.rolling(increase_window).apply(is_increasing))
    country_df.insert(0, 'increasing_avg', country_df.increasing.rolling(increase_window).mean())
    
    pd.set_option('display.max_rows', None)
    display(country_df)
    
    base = alt.Chart(country_df, width=1000).encode(
        alt.X('yearmonthdate(date):T', title='epiweek:O', axis=alt.Axis(tickCount=14, labelAngle=90))
    )

    mark_bar = base.mark_bar().encode(
        x = alt.X('yearmonthdate(date):T', title='epiweek:O', axis=alt.Axis(tickCount=14, labelAngle=90)),
        y = alt.Y('new_cases_smoothed', title='Daily Cases'),
        color = alt.Color('increasing_avg', title=f'{increase_window} Day Increase', scale=alt.Scale(range=['#85C1E9 ', '#F1948A']))
    )

    mark_first_line = base.mark_line(color='#16A085 ').encode(
        y = alt.Y(first_metric)
    )
    
    mark_second_line = base.mark_line(color='#34495E').encode(
        y = alt.Y(second_metric)
    )

    return alt.layer(mark_bar, mark_first_line).resolve_scale(y = 'independent') 


In [ ]:
def choropleth():
    global selected_country
    global first_metric
    global second_metric
    global time_period
    
    countries_df = covid19_df.loc[covid19_df.continent == selected_continent]
    
    increase_window = 5
    
    countries_df.insert(0, 'increasing', countries_df.new_cases_smoothed.rolling(increase_window).apply(is_increasing))
    countries_df.insert(0, 'increasing_avg', countries_df.increasing.rolling(increase_window).mean())
    
    countries_df.to_csv('data.csv')
    
    
    
    countries_df = countries_df.sort_values('date').drop_duplicates('Alpha3',keep='last')
    
    
    
    m = folium.Map(location=[30, -10], zoom_start=3)
    
    countries_geo = f"data/world-countries.json"

    folium.Choropleth(
        geo_data=countries_geo,
        name="choropleth",
        data=countries_df,
        columns=["Alpha3","increasing"],
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.5,
        line_opacity=.1,
        legend_name="",
    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m

In [ ]:
def mobility_data():
    global selected_country
    
    country_df = covid19_df.loc[covid19_df.location == selected_country]
    
    Alpha2 = country_df.iloc[0]['Alpha2']
    
    country_mobility_df = pd.read_csv('data/mobility-reports/2021_' + Alpha2 + '_Region_Mobility_Report.csv')
    
    country_mobility_df = country_mobility_df.drop(columns=['place_id','iso_3166_2_code','census_fips_code'])
    
    return country_mobility_df

In [ ]:
# FILTER COUNTRIES BY CONTINENT

interact(filter_countries, continent = widgets.Dropdown(
    options=continents,
    value='Africa',
    description='Continent',
    disabled=False,
))

# SELECT METRICS

interact(set_first_metric, metric = widgets.Dropdown(
    options=columns,
    value='new_cases_smoothed',
    description='',
    disabled=False,
))

interact(set_second_metric, metric = widgets.Dropdown(
    options=columns,
    value='new_deaths_smoothed',
    description='',
    disabled=False,
))

# PERIOD

interact(set_period, period = widgets.IntText(
    value=100,
    description='Period:',
    disabled=False
))

In [ ]:
display(draw_trend_chart()) # new_cases_smoothed increase/decrease compared to ...
display(draw_chart())
# display(mobility_data())
# display(choropleth())
